In [1]:
import nibabel as nb
print(nb)

<module 'nibabel' from '/usr/local/lib/python3.6/dist-packages/nibabel/__init__.py'>


In [2]:
!pip install nilearn

     |████████████████████████████████| 3.0MB 5.4MB/s 


In [3]:
!pip install pathlib

In [4]:
!pip install transforms3d

     |████████████████████████████████| 71kB 3.6MB/s 
  Created wheel for transforms3d: filename=transforms3d-0.3.1-cp36-none-any.whl size=59374 sha256=aaf024b921d4517bc33e869e983eb399aaf9c9c58d4c718439d5533d004214cd
  Stored in directory: /root/.cache/pip/wheels/3d/3c/84/28d36677f3c760c048bd02b5a547ea0c4027770cc9cdb9af1e
Successfully built transforms3d


In [5]:
from pathlib import Path
from pprint import pprint

import numpy as np
from matplotlib import pyplot as plt
from nilearn import plotting as nlp
import transforms3d
from scipy import ndimage as ndi
import nibabel.testing

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
path = Path('/content/bold.nii.gz')
path

PosixPath('/content/bold.nii.gz')

In [7]:
bold = nb.load(path)

In [15]:
print(bold.shape)

(106, 106, 32, 210)


In [21]:
data = bold.get_fdata
affine = bold.affine
header = bold.header

In [22]:
data

<bound method DataobjImage.get_fdata of <nibabel.nifti1.Nifti1Image object at 0x7f9232a2be80>>

In [23]:
affine

array([[-1.98033774e+00,  5.53158373e-02,  1.87794045e-02,
         1.08459396e+02],
       [ 5.04371151e-02,  1.89977670e+00, -1.12992525e+00,
        -1.06178123e+02],
       [ 2.45448593e-02,  5.59171677e-01,  3.83704519e+00,
        -7.43871536e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [24]:
header

In [28]:
print(bold.ndim)
print(bold.shape)

4
(106, 106, 32, 210)


In [29]:
print(data.shape)
i,j,k = np.array(data.shape)//2
fig, axes = plt.subplots(1,3)
axes[0].imshow(data[i, :, :].T, cmap='Greys_r', origin='lower')
axes[1].imshow(data[:, j, :].T, cmap='Greys_r', origin='lower')
axes[2].imshow(data[:, :, k].T, cmap='Greys_r', origin='lower')

AttributeError: ignored

In [37]:
bold.orthoview()

EOFError: ignored

In [34]:
print(affine)

[[-1.98033774e+00  5.53158373e-02  1.87794045e-02  1.08459396e+02]
 [ 5.04371151e-02  1.89977670e+00 -1.12992525e+00 -1.06178123e+02]
 [ 2.45448593e-02  5.59171677e-01  3.83704519e+00 -7.43871536e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [16]:
T, R, Z, S = transforms3d.affines.decompose44(affine)
print(f"Translation : {T}\nRotation: \n{R} \nZooms: {Z}\nMax shear: {np.abs(S).max()}")

Translation : [ 108.45939636 -106.17812347  -74.38715363]
Rotation: 
[[ 0.9995991   0.02792133  0.00469485]
 [-0.02545874  0.9589349  -0.28248133]
 [-0.01238931  0.28224856  0.95926131]] 
Zooms: [-1.98113198  1.98113208  3.99999994]
Max shear: 3.3332460181366964e-08


In [17]:
Tm, Rm, Zm, Sm = [np.eye(4) for _ in range(4)]
Tm[:3, 3] = T
Rm[:3, :3] = R
Zm[:3, :3] = np.diag(Z)
Sm[[0, 0, 1], [1,2,2]] = S
np.allclose(Tm @ Rm @ Zm @ Sm, affine)


True

In [18]:
print("Orientation: ", nb.aff2axcodes(affine))
print("Zooms: ", nb.affines.voxel_sizes(affine))
print("Obliquity: ", nb.affines.obliquity(affine))

Orientation:  ('L', 'A', 'S')
Zooms:  [1.98113198 1.98113208 3.99999994]
Obliquity:  [0.02831707 0.28757355 0.28642046]
